In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar dataset
df_original = pd.read_csv('raw_diabetes_data_2015.csv')

# Visualizar dataset
df_original

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1,1.0,1.0,b'01202015',b'01',b'20',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,2.0
2,1.0,1.0,b'02012015',b'02',b'01',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
3,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0
4,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441451,72.0,11.0,b'12162015',b'12',b'16',b'2015',1100.0,2.015005e+09,2.015005e+09,NaN,...,4.0,2.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0,2.0
441452,72.0,11.0,b'12142015',b'12',b'14',b'2015',1100.0,2.015005e+09,2.015005e+09,NaN,...,2.0,2.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,1.0
441453,72.0,11.0,b'12232015',b'12',b'23',b'2015',1200.0,2.015005e+09,2.015005e+09,NaN,...,9.0,9.0,3.0,3.0,4.0,9.0,9.0,9.0,9.0,NaN
441454,72.0,11.0,b'12152015',b'12',b'15',b'2015',1100.0,2.015005e+09,2.015005e+09,NaN,...,4.0,2.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,2.0


In [31]:
# --- FUNCIONES DE LIMPIEZA Y TRANSFORMACIÓN ---

# Función para reemplazar valores binarios específicos:
# Convierte 1 en 1 (Sí), 2 en 0 (No), y 7 o 9 en NaN (datos inválidos o faltantes)
def reemplazar_valores_binarios(valor):
    try:
        valor = float(valor)
        if valor == 1:
            return 1
        elif valor == 2:
            return 0
        elif valor in [7, 9]:
            return np.nan
        else:
            return np.nan
    except:
        return np.nan

# Función para convertir códigos de frecuencia (p.ej. 101, 201, 301) en frecuencia diaria estimada:
# 555 se considera 0 (nunca)
# 777, 999, 300 se interpretan como frecuencia muy baja (0.01 veces/día)
# 101-199 → valor - 100 (veces por día)
# 201-299 → (valor - 200) / 7 (promedio semanal convertido a diario)
# 301-399 → (valor - 300) / 30 (promedio mensual convertido a diario)
def convertir_a_veces_dia(valor):
    if pd.isna(valor):
        return np.nan
    valor = int(valor)
    if valor == 555:
        return 0
    if valor in [777, 999, 300]:
        return 0.01
    if 101 <= valor <= 199:
        return valor - 100
    elif 201 <= valor <= 299:
        return (valor - 200) / 7
    elif 301 <= valor <= 399:
        return (valor - 300) / 30
    else:
        return np.nan

# Función para clasificar la frecuencia diaria en categorías cualitativas:
# Convierte veces por día a veces por semana y clasifica en rangos:
# 1 = alta frecuencia (≥7 veces/semana)
# 2 = frecuencia media (3-6.99 veces/semana)
# 3 = frecuencia baja (1-2.99 veces/semana)
# 4 = muy baja o nunca (<1 vez/semana)
def convertir_a_rango(veces_dia):
    if pd.isna(veces_dia):
        return np.nan
    veces_semana = veces_dia * 7
    if veces_semana >= 7:
        return 1  # alto
    elif veces_semana >= 3:
        return 2  # medio
    elif veces_semana >= 1:
        return 3  # bajo
    else:
        return 4  # muy bajo o nunca

# --- VARIABLES SELECCIONADAS ---

# Definición de las columnas a extraer del dataframe original
columnas_seleccionadas = [
    'DIABETE3',
    '_RFHYPE5', 'TOLDHI2', 
    #'BLOODCHO', 
    '_BMI5', 'SMOKE100',
    'CVDSTRK3', 'CVDCRHD4', 'EXERANY2', 'FRUIT1', 'VEGETAB1',
    'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'DIFFWALK', 'SEX',
    '_AGEG5YR', 'EDUCA', 'INCOME2', '_RACEGR3',  
    '_PA30021', '_PASTRNG'              
]

# Crear un dataframe nuevo con solo las columnas seleccionadas
df = df_original[columnas_seleccionadas].copy()

# Diccionario para renombrar columnas con nombres más descriptivos y legibles
mapa_renombrado = {
    'DIABETE3': 'estado_diabetes',
    '_RFHYPE5': 'presion_alta',
    'TOLDHI2': 'colesterol_alto',
    #'BLOODCHO': 'control_colesterol',
    '_BMI5': 'bmi',
    'SMOKE100': 'fumo_100_cigs',
    'CVDSTRK3': 'historial_acv',
    'CVDCRHD4': 'historial_cardiaco',
    'EXERANY2': 'actividad_fisica',
    'FRUIT1': 'frutas_diarias',
    'VEGETAB1': 'verduras_diarias',
    'GENHLTH': 'salud_general',
    'PHYSHLTH': 'dias_mala_salud_fisica',
    'MENTHLTH': 'dias_mala_salud_mental',
    'DIFFWALK': 'dificultad_caminar',
    'SEX': 'sexo',
    '_AGEG5YR': 'grupo_edad',
    'EDUCA': 'nivel_educativo',
    'INCOME2': 'categoria_ingresos',
    '_RACEGR3': 'grupo_racial',
    '_PA30021': 'actividad_300min',
    '_PASTRNG': 'actividad_muscular',
}

# Aplicar renombrado de columnas
df.rename(columns=mapa_renombrado, inplace=True)

# Crear una variable binaria para diabetes:
# 0 = no diabetes (código 1)
# 1 = prediabetes o diabetes (códigos 2 o 3)
df['diabetes_bin'] = df['estado_diabetes'].apply(lambda x: 1 if x in [2, 3] else 0)
df.drop(columns=['estado_diabetes'], inplace=True)  # Eliminar columna original para evitar redundancia

# Ajustar IMC dividiendo por 100 para convertir a escala decimal correcta
df['bmi'] = df['bmi'] / 100

# Aplicar limpieza y codificación binaria en variables relacionadas con presión y colesterol
binarias = ['presion_alta', 'colesterol_alto']
for col in binarias:
    df[col] = df[col].apply(reemplazar_valores_binarios)

# Convertir variables de consumo de frutas y verduras a rangos cualitativos de frecuencia
df['frecuencia_frutas'] = df['frutas_diarias'].apply(convertir_a_veces_dia).apply(convertir_a_rango)
df['frecuencia_verduras'] = df['verduras_diarias'].apply(convertir_a_veces_dia).apply(convertir_a_rango)
# Eliminar las columnas originales de frutas y verduras
df.drop(columns=['frutas_diarias', 'verduras_diarias'], inplace=True)

# Limpiar variables de actividad física: reemplazar códigos inválidos 7 y 9 por NaN
df['actividad_300min'] = df['actividad_300min'].replace({7: np.nan, 9: np.nan})
df['actividad_muscular'] = df['actividad_muscular'].replace({7: np.nan, 9: np.nan})

# Imputar valores faltantes en variables clave para evitar pérdida excesiva de filas:
# bmi → mediana para valores faltantes
# actividad_fisica → 2 (asumiendo código 2 = no realiza actividad)
# actividad_300min y actividad_muscular → 0 (no cumple actividad)
df['bmi'] = df['bmi'].fillna(df['bmi'].median())
df['actividad_fisica'] = df['actividad_fisica'].fillna(2)
df['actividad_300min'] = df['actividad_300min'].fillna(0)
df['actividad_muscular'] = df['actividad_muscular'].fillna(0)

# Mostrar resumen de valores faltantes tras imputación
print("Valores faltantes por columna después de imputar:")
print(df.isnull().sum())

# Eliminar filas que aún tengan valores faltantes (NaN) después de imputar
df.dropna(inplace=True)

print(f"Dimensiones finales del dataframe: {df.shape}")
print(df.head())

# Eliminar filas duplicadas y reiniciar índice
df = df.drop_duplicates().reset_index(drop=True)

# Guardar dataframe limpio a archivo CSV
df.to_csv('cleaned_diabetes_data_extended.csv', index=False)


Valores faltantes por columna después de imputar:
presion_alta               1367
colesterol_alto           62715
bmi                           0
fumo_100_cigs             14255
historial_acv                 0
historial_cardiaco            1
actividad_fisica              0
salud_general                 2
dias_mala_salud_fisica        1
dias_mala_salud_mental        0
dificultad_caminar        12334
sexo                          0
grupo_edad                    0
nivel_educativo               0
categoria_ingresos         3301
grupo_racial                  0
actividad_300min              0
actividad_muscular            0
diabetes_bin                  0
frecuencia_frutas         29150
frecuencia_verduras       33138
dtype: int64
Dimensiones finales del dataframe: (351178, 21)
   presion_alta  colesterol_alto    bmi  fumo_100_cigs  historial_acv  \
0           0.0              1.0  40.18            1.0            2.0   
1           1.0              0.0  25.09            1.0            2.0  

In [32]:
# Visualizar Dataset Final
df

,presion_alta,colesterol_alto,bmi,fumo_100_cigs,historial_acv,historial_cardiaco,actividad_fisica,salud_general,dias_mala_salud_fisica,dias_mala_salud_mental,...,sexo,grupo_edad,nivel_educativo,categoria_ingresos,grupo_racial,actividad_300min,actividad_muscular,diabetes_bin,frecuencia_frutas,frecuencia_verduras
0,0.0,1.0,40.18,1.0,2.0,2.0,2.0,5.0,15.0,18.0,...,2.0,9.0,4.0,3.0,1.0,2.0,2.0,1,3.0,1.0
1,1.0,0.0,25.09,1.0,2.0,2.0,1.0,3.0,88.0,88.0,...,2.0,7.0,6.0,1.0,1.0,2.0,2.0,1,3.0,4.0
2,0.0,1.0,28.19,2.0,2.0,2.0,2.0,5.0,30.0,30.0,...,2.0,9.0,4.0,8.0,1.0,2.0,2.0,1,1.0,3.0
3,1.0,0.0,24.37,2.0,2.0,2.0,2.0,5.0,20.0,88.0,...,2.0,9.0,5.0,77.0,1.0,2.0,2.0,1,1.0,2.0
4,0.0,0.0,26.52,2.0,2.0,2.0,1.0,2.0,88.0,88.0,...,2.0,11.0,3.0,6.0,1.0,1.0,2.0,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349918,0.0,1.0,45.00,2.0,2.0,2.0,2.0,3.0,5.0,88.0,...,1.0,5.0,6.0,7.0,5.0,2.0,2.0,1,2.0,2.0
349919,0.0,1.0,18.42,2.0,2.0,2.0,2.0,4.0,88.0,88.0,...,2.0,11.0,2.0,4.0,5.0,2.0,2.0,0,4.0,3.0
349920,1.0,0.0,28.34,2.0,2.0,2.0,1.0,1.0,88.0,88.0,...,2.0,2.0,5.0,2.0,5.0,2.0,2.0,1,1.0,3.0
349921,0.0,0.0,23.15,2.0,2.0,2.0,2.0,3.0,88.0,88.0,...,1.0,7.0,5.0,1.0,5.0,2.0,2.0,1,1.0,4.0
